In [1]:
import pickle
with open('Clean-dataset-1.pkl','rb') as fp:
    df = pickle.load(fp)
df.shape

(24053764, 4)

In [2]:
df = df.sort_values(by='Date')

df = df.drop(axis=1,labels=['Date'])

In [3]:
import scipy as np
CID = np.array(df.CustomerID)
MID = np.array(df.MovieID)
Score = np.array(df.Score)
l = int(df.shape[0]*0.7)

In [4]:
tr_data = {}
for i in range(l):
    r = CID[i]; c = MID[i]
    tr_data[(r,c)] = Score[i]

In [5]:
ts_data = {}
for i in range(l,CID.shape[0]):
    r = CID[i]; c = MID[i]
    ts_data[(r,c)] = Score[i]

In [6]:
row = CID.max()+1
col = MID.max()+1
del df,MID,CID,Score

In [7]:
import gc; gc.collect()

7

In [11]:
import numpy
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        count = 0
        for tup in R.keys():
            i,j = tup
            eij = R[(i,j)] - numpy.dot(P[i,:],Q[:,j])
            for k in range(K):
                tmp_1 = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                tmp_2 = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
                P[i][k] = tmp_1; Q[k][j] = tmp_2
        #eR = numpy.dot(P,Q)
        e = 0
        for tup in R.keys():
            i,j = tup
            e = e + pow(R[(i,j)] - numpy.dot(P[i,:],Q[:,j]), 2)
            for k in range(K):
                e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break

    return P, Q.T

In [ ]:
if __name__ == "__main__":
    R = tr_data

    #R = numpy.array(R)

    N = row
    M = col
    K = 100

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K)

In [15]:
list(tr_data.keys())[:10]

[(91061, 2865),
 (91061, 1797),
 (91061, 3869),
 (91061, 1366),
 (91061, 3729),
 (91061, 2477),
 (91061, 2947),
 (91061, 3420),
 (21742, 606),
 (21742, 1366)]